In [1]:
import pandas as pd
import re
import json

# Load the dataset
file_path = 'adzuna_data_related_jobs.csv'
df = pd.read_csv(file_path)

# Step 1: Drop the 'Company Size' column which is entirely NaN
df_cleaned = df.drop(columns=['Company Size'])

# Step 2: Rename the column 'Required Language' to 'Description'
df_cleaned.rename(columns={'Required Language': 'Description'}, inplace=True)

# Step 3: Define functions to parse the 'Description' field
def extract_language_requirements(description):
    # Look for human language mentions (e.g., English, French, etc.)
    language_keywords = re.findall(r'\b(English|French|German|Spanish|Chinese|Dutch|Portuguese|Italian|Japanese|Korean|Russian|Hindi)\b', description, re.IGNORECASE)
    return ', '.join(set(language_keywords)) if language_keywords else 'N/A'

def extract_company_info(description):
    # This function tries to find information related to company specifics or benefits.
    if 'company' in description.lower() or 'startup' in description.lower() or 'global' in description.lower():
        return "Details found in description"
    return 'N/A'

def extract_soft_skills(description):
    # Look for common soft skills (e.g., communication, leadership, teamwork, etc.)
    soft_skills_keywords = re.findall(r'\b(communication|leadership|teamwork|collaboration|problem-solving|creativity|adaptability|flexibility|time management|empathy|negotiation|critical thinking|conflict resolution|interpersonal skills|decision making|work ethic|stress management)\b', description, re.IGNORECASE)
    return ', '.join(set(soft_skills_keywords)) if soft_skills_keywords else 'N/A'

def extract_data_skills(description):
    # Look for data-related skills (e.g., Python, Spark, SQL, etc.)
    data_skills_keywords = re.findall(r'\b(Python|Java|C\+\+|JavaScript|SQL|R|Scala|AWS|Azure|GCP|Spark|Hadoop|Tableau|Power BI|Excel|Pandas|NumPy|TensorFlow|PyTorch|Keras|MATLAB|Snowflake|Docker|Kubernetes)\b', description, re.IGNORECASE)
    return ', '.join(set(data_skills_keywords)) if data_skills_keywords else 'N/A'

# Step 4: Drop duplicate rows if any exist and reset index
df_cleaned = df_cleaned.drop_duplicates().reset_index(drop=True)

# Step 5: Ensure all column names are unique
df_cleaned = df_cleaned.loc[:, ~df_cleaned.columns.duplicated()]

# Step 6: Re-apply the extraction functions using the correct method
df_cleaned['Language Requirements'] = df_cleaned['Description'].apply(lambda desc: extract_language_requirements(str(desc)))
df_cleaned['Company Info'] = df_cleaned['Description'].apply(lambda desc: extract_company_info(str(desc)))
df_cleaned['Soft Skills'] = df_cleaned['Description'].apply(lambda desc: extract_soft_skills(str(desc)))
df_cleaned['Data Skills'] = df_cleaned['Description'].apply(lambda desc: extract_data_skills(str(desc)))

# Step 7: Save the cleaned dataset to a new CSV file
df_cleaned.to_csv('adzuna_data_cleaned.csv', index=False)

# Step 8: Save all operations to a JSON file
operations = [
    {"step": 1, "action": "Drop column", "details": "Dropped 'Company Size' column which is entirely NaN"},
    {"step": 2, "action": "Rename column", "details": "Renamed 'Required Language' to 'Description'"},
    {"step": 3, "action": "Define extraction functions", "details": "Defined functions to extract language requirements, company info, soft skills, and data skills from the 'Description'"},
    {"step": 4, "action": "Drop duplicates", "details": "Dropped duplicate rows and reset index"},
    {"step": 5, "action": "Ensure unique columns", "details": "Ensured all column names are unique"},
    {"step": 6, "action": "Apply extraction functions", "details": "Applied extraction functions to 'Description' column to create new columns for 'Language Requirements', 'Company Info', 'Soft Skills', and 'Data Skills'"},
    {"step": 7, "action": "Save CSV", "details": "Saved the cleaned dataset to a new CSV file 'adzuna_data_cleaned.csv'"}
]

with open('adzuna_cleaning_operations.json', 'w') as f:
    json.dump(operations, f, indent=4)

# Step 9: Display the cleaned dataset to the user
print(df_cleaned.head())

  Country                      Job Title         Company  \
0      fr           Head of Data Science         Aircall   
1      fr  Cloud Engineer - Data Science         Lenstra   
2      fr       Head Of Data Science H/F       HelloWork   
3      fr       Head Of Data Science F/H  RFC Consulting   
4      fr       Head Of Data Science H/F       HelloWork   

               Industry Job Type Remote Type  Salary Min  Salary Max  \
0  Emplois Informatique      NaN         NaN    125000.0    150000.0   
1               Unknown      NaN   full_time     45000.0     55000.0   
2  Emplois Informatique      NaN         NaN    100000.0    125000.0   
3  Emplois Informatique      NaN         NaN    100000.0    125000.0   
4               Unknown      NaN   full_time     70000.0     70000.0   

   Experience             Post Date  \
0         NaN  2024-07-19T08:35:28Z   
1         NaN  2024-07-30T23:53:58Z   
2         NaN  2024-10-11T11:42:53Z   
3         NaN  2023-10-22T15:49:17Z   
4         N

  Country                      Job Title         Company  \
0      fr           Head of Data Science         Aircall   
1      fr  Cloud Engineer - Data Science         Lenstra   
2      fr       Head Of Data Science H/F       HelloWork   
3      fr       Head Of Data Science F/H  RFC Consulting   
4      fr       Head Of Data Science H/F       HelloWork   

               Industry Job Type Remote Type  Salary Min  Salary Max  \
0  Emplois Informatique      NaN         NaN    125000.0    150000.0   
1               Unknown      NaN   full_time     45000.0     55000.0   
2  Emplois Informatique      NaN         NaN    100000.0    125000.0   
3  Emplois Informatique      NaN         NaN    100000.0    125000.0   
4               Unknown      NaN   full_time     70000.0     70000.0   

   Experience             Post Date  \
0         NaN  2024-07-19T08:35:28Z   
1         NaN  2024-07-30T23:53:58Z   
2         NaN  2024-10-11T11:42:53Z   
3         NaN  2023-10-22T15:49:17Z   
4         N

In [3]:
import pandas as pd
from googletrans import Translator
from tqdm import tqdm
import time

# 加载数据
data = pd.read_csv('adzuna_data_cleaned.csv')
translator = Translator()

# 缓存翻译结果
translation_cache = {}

# 定义翻译函数
def translate_text(text):
    if pd.isnull(text):
        return text
    if text in translation_cache:
        return translation_cache[text]
    try:
        translated = translator.translate(text, src='auto', dest='en').text
        translation_cache[text] = translated  #
        return translated
    except Exception as e:
        return str(e)

# 将描述按频率排序，以减少重复调用
data['Description'] = data['Description'].fillna('')
descriptions_sorted = data['Description'].value_counts().index.tolist()

# 逐一翻译并显示进度条
translated_texts = {}
for desc in tqdm(descriptions_sorted, desc="Translating unique descriptions"):
    translated_texts[desc] = translate_text(desc)

# 将翻译结果应用到整个 DataFrame
data['Description_English'] = data['Description'].map(translated_texts)

# 保存结果
data.to_csv('translated_jobs.csv', index=False)
print("Translation completed.")

Translating unique descriptions: 100%|██████████| 3855/3855 [28:14<00:00,  2.27it/s] 


Translation completed.


In [4]:
import pandas as pd
from googletrans import Translator  # install with: pip install googletrans==4.0.0-rc1
import re

# Load the dataset
file_path = 'translated_jobs.csv'
data = pd.read_csv(file_path)

# Helper functions for parsing
def extract_experience(description):
    experience_levels = ['Internship', 'Junior', 'Senior']
    for level in experience_levels:
        if re.search(level, description, re.IGNORECASE):
            return level
    return 'Unspecified'

def extract_soft_skills(description):
    soft_skills = ['communication', 'teamwork', 'leadership', 'problem-solving', 'adaptability']
    found_skills = [skill for skill in soft_skills if re.search(skill, description, re.IGNORECASE)]
    return found_skills if found_skills else None

def extract_data_skills(description):
    data_skills = ['SQL', 'Python', 'AWS', 'R', 'Machine Learning', 'Data Analysis']
    found_skills = [skill for skill in data_skills if re.search(skill, description, re.IGNORECASE)]
    return found_skills if found_skills else None

def extract_language_requirements(description):
    languages = ['English', 'French', 'German', 'Dutch']
    found_languages = [lang for lang in languages if re.search(lang, description, re.IGNORECASE)]
    return found_languages if found_languages else None

def extract_job_requirements(description):
    # Custom extraction for any specific requirements like "3+ years experience" or "Master's degree"
    requirements = re.findall(r'\d+\+? years|Bachelor|Master|PhD', description, re.IGNORECASE)
    return requirements if requirements else None

# Applying extraction functions
data['Experience_Type'] = data['Description_English'].apply(extract_experience)
data['Soft_Skills'] = data['Description_English'].apply(extract_soft_skills)
data['Data_Skills'] = data['Description_English'].apply(extract_data_skills)
data['Language_Requirements'] = data['Description_English'].apply(extract_language_requirements)
data['Additional_Requirements'] = data['Description_English'].apply(extract_job_requirements)

# Save or display the processed data
data.to_csv('translated_and_categorized_jobs.csv', index=False)
print(data[['Description_English', 'Experience_Type', 'Soft_Skills', 'Data_Skills', 'Language_Requirements', 'Additional_Requirements']].head())

                                 Description_English Experience_Type  \
0  Aircall is a place where voices are valued. Ba...     Unspecified   
1  We are currently seeking a talented AWS Cloud ...     Unspecified   
2  The missions of the HELLOWORK Group position i...     Unspecified   
3  We are looking for our client on behalf of our...     Unspecified   
4  Hellowork continues its ascent and our ambitio...     Unspecified   

       Soft_Skills Data_Skills Language_Requirements Additional_Requirements  
0  [communication]         [R]                  None                    None  
1             None    [AWS, R]                  None                    None  
2             None         [R]              [French]                    None  
3             None         [R]                  None               [3 years]  
4             None         [R]                  None                    None  


In [9]:
file_path = 'translated_and_categorized_jobs.csv'

data = pd.read_csv(file_path)



# Display the first few rows of the dataset to understand the columns and structure

data.head()

,Country,Job Title,Company,Industry,Job Type,Remote Type,Salary Min,Salary Max,Experience,Post Date,...,Language Requirements,Company Info,Soft Skills,Data Skills,Description_English,Experience_Type,Soft_Skills,Data_Skills,Language_Requirements,Additional_Requirements
0,fr,Head of Data Science,Aircall,Emplois Informatique,NaN,NaN,125000.0,150000.0,NaN,2024-07-19T08:35:28Z,...,NaN,NaN,communication,NaN,Aircall is a place where voices are valued. Ba...,Unspecified,['communication'],['R'],NaN,NaN
1,fr,Cloud Engineer - Data Science,Lenstra,Unknown,NaN,full_time,45000.0,55000.0,NaN,2024-07-30T23:53:58Z,...,NaN,Details found in description,NaN,AWS,We are currently seeking a talented AWS Cloud ...,Unspecified,NaN,"['AWS', 'R']",NaN,NaN
2,fr,Head Of Data Science H/F,HelloWork,Emplois Informatique,NaN,NaN,100000.0,125000.0,NaN,2024-10-11T11:42:53Z,...,NaN,NaN,NaN,NaN,The missions of the HELLOWORK Group position i...,Unspecified,NaN,['R'],['French'],NaN
3,fr,Head Of Data Science F/H,RFC Consulting,Emplois Informatique,NaN,NaN,100000.0,125000.0,NaN,2023-10-22T15:49:17Z,...,NaN,NaN,NaN,NaN,We are looking for our client on behalf of our...,Unspecified,NaN,['R'],NaN,['3 years']
4,fr,Head Of Data Science H/F,HelloWork,Unknown,NaN,full_time,70000.0,70000.0,NaN,2024-10-09T05:38:41Z,...,NaN,NaN,NaN,NaN,Hellowork continues its ascent and our ambitio...,Unspecified,NaN,['R'],NaN,NaN


In [36]:
# Step 1: Reload the original data to retain all entries for a fresh start
data = pd.read_csv(file_path)

# Step 2: Remove columns with high missing values but retain as many rows as possible by filling missing values
# We will fill 'Salary Min' and 'Salary Max' missing values with the median values

# Define a dictionary to map country codes to full country names
country_mapping = {
    'fr': 'France',
    'gb': 'United Kingdom',
    'de': 'Germany',
    'nl': 'Netherlands',
    'us': 'United States',
    'ca': 'Canada',
    'au': 'Australia',
    'es': 'Spain',
    'it': 'Italy',
    'be': 'Belgium',
    'ch': 'Switzerland',
    'at': 'Austria',
    'ie': 'Ireland',
    'in': 'India',
    'sg': 'Singapore',
    'hk': 'Hong Kong'
}

# Map country codes to full names in the 'Country' column
data['Country'] = data['Country'].map(country_mapping).fillna(data['Country'])


# Fill Salary Min and Salary Max with their respective medians
data['Salary Min'].fillna(data['Salary Min'].median(), inplace=True)
data['Salary Max'].fillna(data['Salary Max'].median(), inplace=True)

# Convert 'Post Date' to datetime for consistency
data['Post Date'] = pd.to_datetime(data['Post Date'], errors='coerce')

data['Post Date'] = data['Post Date'].dt.strftime('%Y-%m-%d')

# For columns with categorical missing data (e.g., 'Soft Skills', 'Data Skills', 'Language Requirements'), we'll fill missing values with 'Not specified'
data['Soft Skills'].fillna('Not specified', inplace=True)
data['Data Skills'].fillna('Not specified', inplace=True)
data['Language Requirements'].fillna('Not specified', inplace=True)
data['Additional_Requirements'].fillna('Not specified', inplace=True)

# To retain rows, we'll keep all entries with these fillers instead of removing rows
data = data.drop(columns=['Data Skills', 'Experience_Type', 'Soft Skills', 'Company Info', 'Additional_Requirements', 'Remote Type', 'Experience', 'Description'])


# Save the cleaned data to a new file
output_path = 'translated_and_filled_jobs.csv'
data.to_csv(output_path, index=False)

# Display a summary to confirm the number of rows and structure after filling missing values
data.info(), output_path

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7706 entries, 0 to 7705
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Country                7706 non-null   object 
 1   Job Title              7706 non-null   object 
 2   Company                7683 non-null   object 
 3   Industry               7706 non-null   object 
 4   Job Type               1955 non-null   object 
 5   Salary Min             7706 non-null   float64
 6   Salary Max             7706 non-null   float64
 7   Post Date              7706 non-null   object 
 8   Redirect URL           7706 non-null   object 
 9   Location               7706 non-null   object 
 10  Job Category           7706 non-null   object 
 11  Language Requirements  7706 non-null   object 
 12  Description_English    7706 non-null   object 
 13  Soft_Skills            218 non-null    object 
 14  Data_Skills            7706 non-null   object 
 15  Lang

/tmp/ipykernel_16230/293898178.py:32: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Salary Min'].fillna(data['Salary Min'].median(), inplace=True)
/tmp/ipykernel_16230/293898178.py:33: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value

(None, 'translated_and_filled_jobs.csv')

In [37]:
data

,Country,Job Title,Company,Industry,Job Type,Salary Min,Salary Max,Post Date,Redirect URL,Location,Job Category,Language Requirements,Description_English,Soft_Skills,Data_Skills,Language_Requirements
0,France,Head of Data Science,Aircall,Emplois Informatique,NaN,125000.0,150000.0,2024-07-19,https://www.adzuna.fr/details/4786334013?utm_m...,"Paris, Ile-de-France",data science,Not specified,Aircall is a place where voices are valued. Ba...,['communication'],['R'],NaN
1,France,Cloud Engineer - Data Science,Lenstra,Unknown,NaN,45000.0,55000.0,2024-07-30,https://www.adzuna.fr/details/4801694497?utm_m...,"9ème Arrondissement, Paris",data science,Not specified,We are currently seeking a talented AWS Cloud ...,NaN,"['AWS', 'R']",NaN
2,France,Head Of Data Science H/F,HelloWork,Emplois Informatique,NaN,100000.0,125000.0,2024-10-11,https://www.adzuna.fr/details/4898580210?utm_m...,"Ille-et-Vilaine, Bretagne",data science,Not specified,The missions of the HELLOWORK Group position i...,NaN,['R'],['French']
3,France,Head Of Data Science F/H,RFC Consulting,Emplois Informatique,NaN,100000.0,125000.0,2023-10-22,https://www.adzuna.fr/details/4386856071?utm_m...,"Aix-en-Provence, Bouches-du-Rhône",data science,Not specified,We are looking for our client on behalf of our...,NaN,['R'],NaN
4,France,Head Of Data Science H/F,HelloWork,Unknown,NaN,70000.0,70000.0,2024-10-09,https://www.adzuna.fr/details/4894937419?utm_m...,"Ille-et-Vilaine, Bretagne",data science,Not specified,Hellowork continues its ascent and our ambitio...,NaN,['R'],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7701,France,"GenAI and AIML Solutions Architect, AGS France...",AWS EMEA SARL,Unknown,NaN,50000.0,56000.0,2024-10-10,https://www.adzuna.fr/details/4897861227?utm_m...,"Courbevoie, Nanterre",machine learning engineer,Not specified,Are you passionate about Artificial Intelligen...,NaN,"['AWS', 'R', 'Machine Learning']",NaN
7702,France,"GenAI and AIML Solutions Architect, AGS France...",Amazon,Unknown,NaN,50000.0,56000.0,2024-10-11,https://www.adzuna.fr/details/4899256592?utm_m...,"Courbevoie, Nanterre",machine learning engineer,Not specified,Description Are you passionate about Artificia...,NaN,"['AWS', 'R', 'Machine Learning']",NaN
7703,France,Senior Data Scientist H/F,Alstom,Unknown,NaN,50000.0,56000.0,2024-09-25,https://www.adzuna.fr/details/4875386687?utm_m...,"Selommes, Vendôme",machine learning engineer,Not specified,"Req ID :462283 At Alstom, we understand transp...",NaN,['R'],NaN
7704,France,"GenAI and AIML Solutions Architect, AGS France...",AWS EMEA SARL,Unknown,NaN,50000.0,56000.0,2024-10-10,https://www.adzuna.fr/details/4897861069?utm_m...,"Courbevoie, Nanterre",machine learning engineer,Not specified,Are you passionate about Artificial Intelligen...,NaN,"['AWS', 'R', 'Machine Learning']",NaN
